# Geoqueries en Mongo

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#¿Qué-podemos-hacer-con-las-geoqueries?" data-toc-modified-id="¿Qué-podemos-hacer-con-las-geoqueries?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>¿Qué podemos hacer con las geoqueries?</a></span></li><li><span><a href="#Generamos-la-conexión-con-Mongo" data-toc-modified-id="Generamos-la-conexión-con-Mongo-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generamos la conexión con Mongo</a></span></li><li><span><a href="#Importar-colecciones" data-toc-modified-id="Importar-colecciones-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Importar colecciones</a></span></li><li><span><a href="#Crear-indexes" data-toc-modified-id="Crear-indexes-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Crear indexes</a></span></li><li><span><a href="#Generamos-las-conexiones-con-la-base-de-datos" data-toc-modified-id="Generamos-las-conexiones-con-la-base-de-datos-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Generamos las conexiones con la base de datos</a></span></li><li><span><a href="#Empezamos-con-las-Geoqueries" data-toc-modified-id="Empezamos-con-las-Geoqueries-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Empezamos con las Geoqueries</a></span></li><li><span><a href="#Intersects" data-toc-modified-id="Intersects-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Intersects</a></span></li><li><span><a href="#GeoWithin" data-toc-modified-id="GeoWithin-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>GeoWithin</a></span></li><li><span><a href="#Near" data-toc-modified-id="Near-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Near</a></span></li></ul></div>

## ¿Qué podemos hacer con las geoqueries?
Con las geoquerías podemos responder a preguntas como las siguientes
 * ¿Dónde están las farmacias más cercanas a mi ubicación?
 * ¿Qué restaurantes hay en esta poligonal?

Cada punto de la Tierra se caracteriza por dos números:
 * Longitud: ángulo respecto al meridiano de Greenwich. Va de -180° (antiGreenwich) a +180° (también antiGreenwich)
 * Latitud: ángulo con respecto al ecuador. Va de -90° (sur) a +90° (norte)

![latierra](../images/latlon.gif)

## Generamos la conexión con Mongo

In [2]:
from pymongo import MongoClient

In [5]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

## Importar colecciones     
Sigamos la documentación oficial de Mongo: [Mongo Geo Example](https://docs.mongodb.com/manual/tutorial/geospatial-tutorial/)
Creamos estas dos colecciones nuevas:
 * `restaurants2`
 * `neighborhoods`

In [6]:
rest = db.get_collection("rest")
barr = db.get_collection("barrios")

## Crear indexes
La indexación geoespacial de MongoDB permite ejecutar eficientemente **consultas espaciales** sobre una colección que contiene formas y puntos geoespaciales.
Vamos a crear un índice geográfico para que Mongo sepa que queremos hacer geo-consultas en esta colección.  
Esto sólo debe hacerse una vez, y ahora la colección de Mongo está correctamente indexada para siempre.       
[Documentación](https://docs.mongodb.com/manual/geospatial-queries/#geospatial-indexes) de los índices.

In [7]:
from pymongo import GEOSPHERE

In [9]:
# Creamos el  index 2dsphere desde pymongo
db.barrios.create_index([("geometry", GEOSPHERE)])

'geometry_2dsphere'

Es importante indicarle el campo donde tenemos tanto el tipo de elemento (polígonos o  puntos (point)) como las coordenadas.

In [10]:
"""
Si quiero crear el de restaurants LE TENGO QUE DECIR CUÁL ES EL CAMPO DÓNDE ESTÁ LA info no solo de las
coordenadas si no del tipo de dato que es, polígono, multipolígono....en este caso está dentro del campo location
en la colección de barrios está dentro de un campo llamado geometry
"""
db.rest.create_index([("location", GEOSPHERE)])

'location_2dsphere'

## Empezamos con las Geoqueries   
¿En qué barrio estoy?      
¿Qué elementos **intersecan** el elemento dado?

## Intersects

Necesitamos tener los datos  con tipo polígono en este caso tenemos una colección con barrios donde cada barrio  tiene dentro  el polígono que lo delimita. Y cada barrio es un documento, por eso cuando  intersecta me devuelve el documento y yo veo  que es el barrio  en concreto.    
Selecciona los documentos cuyos datos geoespaciales se cruzan con un objeto GeoJSON especificado; es decir, cuando la intersección de los datos y el objeto especificado no está vacía.

In [11]:
def type_point(lista):
    return {"type":"Point", "coordinates": lista}

In [12]:
coordenadas = [-73.93, 40.82]

In [13]:
barr.find_one().keys()

dict_keys(['_id', 'geometry', 'name'])

El geometry que va sin dolar es porque vamos a buscar en la colección de barrios y donde está la información es dentro de la key geometry, se ve justo aquí arriba 👆🏻. El geometry que lleva dolar es la sintaxis de mongo para todas las queries geoespaciales. 
Para hacer una query geoespacial a Mongo tengo que pasarle las coordenadas en tipo point

In [14]:
punto = type_point(coordenadas)
punto

{'type': 'Point', 'coordinates': [-73.93, 40.82]}

In [16]:
proy = {"_id":0, "name":1}

In [17]:
barr.find_one({"geometry": {"$geoIntersects": {"$geometry": punto}}}, proy)

{'name': 'West Concourse'}

In [ ]:
# LO QUE LLAMÁBAMOS FILTRO A LA HORA DE HACER LA QUERY


## GeoWithin

¿Qué elementos están **contenidos** en el elemento dado?     
¿Qué restaurantes hay en este barrio (polígono)?

In [ ]:
# Vemos la estructura de los documentos sacando uno y viendo sus keys


In [ ]:
#bUSCAMOS UN barrio en  concreto


In [ ]:
#Extraigo las coordenadas


In [ ]:
#Saco las keys de un documento de la coleccción  de restaurantes


In [ ]:
# ER FIRTRO
#Location porque la key de la colección donde están las coordenadas se llama location

In [ ]:
df.head()

## Near

¿Qué elementos se encuentran a una distancia de una geometría determinada?     
`$nearSphere`/`$near` y `$maxDistance` nos ayudan a encontrar todos los elementos dentro de `maxDistance` metros de la geometría deseada, **ordenados** de más cercano a más lejano.

In [ ]:
#  Mi posición, voy a buscar restaurantes cercanos a  ESTE punto


In [ ]:
# Km que quiero  caminar


Probamos geoqueries, desde el punto en el que estamos u otros puntos con las funciones que tenemos en el otro jupyter....